In [3]:
import folium
import pandas as pd

# Import data
%store -r data_sampah
%store -r koordinat_BDG

# Mengubah kolom tanggal menjadi tipe data datetime
data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'], format='%m/%d/%Y')

# Membuat kolom baru untuk mengelompokkan kolom tanggal berdasarkan tahun
data_sampah['tahun'] = data_sampah['tanggal'].dt.to_period('Y')

# Mengelompokkan berdasarkan tahun dan kecamatan, kemudian menghitung jumlah sampah
grouped_data = data_sampah.groupby(['tahun', 'kecamatan']).agg({
    'jumlah_sampah': 'sum'
}).reset_index()

# Menggabungkan dengan data koordinat dan data sampah
merged_data = pd.merge(grouped_data, koordinat_BDG, on='kecamatan')

# Memasukan longitude dan latitude (rata-rata supaya berada ditengah)
bandung_map = folium.Map(location=[koordinat_BDG['Lintang Selatan'].mean(), koordinat_BDG['Bujur Timur'].mean()], zoom_start=13)

# Menambahkan Data ke Peta
for index, row in merged_data.iterrows():
    # Menentukan warna marker berdasarkan jumlah sampah
    if row['jumlah_sampah'] > 1500:
        color = 'red'
    elif row['jumlah_sampah'] > 1000:
        color = 'orange'
    else:
        color = 'green'

    # Membuat HTML untuk Tooltip
    tooltip_html = f"""
    <h4>{row['kecamatan']}</h4>
    <h5>Tahun: {row['tahun']}</h5>
    <h5>Jumlah Sampah: {row['jumlah_sampah']} Ton</h5>
    """

    # Menambahkan marker untuk setiap baris dengan warna berdasarkan jumlah sampah
    marker = folium.Marker(
        location=[row['Lintang Selatan'], row['Bujur Timur']],
        tooltip=tooltip_html,
        icon=folium.Icon(color=color),
    )
    marker.add_to(bandung_map)

# Menampilkan peta di notebook
display(bandung_map)

%store bandung_map


Stored 'bandung_map' (Map)
